In [30]:
from cred import GUARDIAN_KEY
import requests
import pandas as pd

## Understanding APIs
When we visit websites online, we provide an address. The address specifies what website data we want to be sent back from the server for us to view in our browsers.

A simple way of thinking about an API, is to also think of it as a website that sends us data which varies depending on what address we provide it. The address has to be a little more complicated than simply www.google.com, but as long as we build that address correctly, we will get what we ask for.

### Guardian API - Interactive Exploration
The Guardian API provides a helpful tool for us to explore how the address it built, and what results we can get back. It is also useful in showing us what kind of options we might have when requesting data.

[Explore the Guardian API](https://open-platform.theguardian.com/explore/)

## Communicating with the API in Python
So we can see how the web address is built using the API explorer, but how do we build that address using Python, and communicate with the API server so that it sends us back the data?

### A Very Basic Example
Initially we will just make the simplest query we can, which is simply contacting the API with our credential key to get it to send back *something*.

First we define the end point, which is essentially the root of the address we are going to start with. The Guardian API has a [few different endpoints](https://open-platform.theguardian.com/documentation/) but for our purposes, the *content* end point is the one we need.

In [2]:
API_ENDPOINT = 'http://content.guardianapis.com/search'

In [41]:
# Remember the API key you were given. It is simply a string of numbers and letters.
GUARDIAN_KEY

'***REMOVED***'

In [42]:
# We first create a dictionary that has a parameter name of api-key, and then our key as its value.
parameters = {'api-key':GUARDIAN_KEY}

parameters

{'api-key': '***REMOVED***'}

Now we're going to communicate with the Guardian API using `requests`. We will pass in the address we are going to communicate with, the `API_ENDPOINT` and by providing `requests` with a dictionary of parameters, it can build the rest of the address for us before making its request for data.

In [43]:
response = requests.get(API_ENDPOINT, params=parameters)

Requests has now communicated with the server and whatever the server sent back has been packaged up in a special `Response` object.

In [50]:
# We can see the type of object
type(response)

requests.models.Response

In [51]:
# and if we look at the object itself it doesn't tell us much.
response

<Response [200]>

In [52]:
# One useful check is to see how requests built the url for us...

response.url

'http://content.guardianapis.com/search?api-key=***REMOVED***'

Finally, we can see the data that was sent back by asking the response object to show us its data in JSON format.
JSON is essentially a set of nested dictionaries.

In [53]:
response.json()

{'response': {'status': 'ok',
  'userTier': 'developer',
  'total': 2405810,
  'startIndex': 1,
  'pageSize': 10,
  'currentPage': 1,
  'pages': 240581,
  'orderBy': 'newest',
  'results': [{'id': 'politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
    'type': 'article',
    'sectionId': 'politics',
    'sectionName': 'Politics',
    'webPublicationDate': '2023-02-15T15:54:11Z',
    'webTitle': 'Brussels contacts Sweden over plans to deport British woman with Alzheimer’s',
    'webUrl': 'https://www.theguardian.com/politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
    'apiUrl': 'https://content.guardianapis.com/politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
    'isHosted': False,
    'pillarId': 'pillar/news',
    'pillarName': 'News'},
   {'id': 'uk-news/2023/feb/15/lucy-letby-nurse-accused-of-killing-babies-entered-grand-national-on-day-of-attempted',
   

In [59]:
# The top level dictionary just has one key called 'response' which contains all the other information.
guardian_data = response.json()['response']
guardian_data

{'status': 'ok',
 'userTier': 'developer',
 'total': 2405810,
 'startIndex': 1,
 'pageSize': 10,
 'currentPage': 1,
 'pages': 240581,
 'orderBy': 'newest',
 'results': [{'id': 'politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
   'type': 'article',
   'sectionId': 'politics',
   'sectionName': 'Politics',
   'webPublicationDate': '2023-02-15T15:54:11Z',
   'webTitle': 'Brussels contacts Sweden over plans to deport British woman with Alzheimer’s',
   'webUrl': 'https://www.theguardian.com/politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
   'apiUrl': 'https://content.guardianapis.com/politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
   'isHosted': False,
   'pillarId': 'pillar/news',
   'pillarName': 'News'},
  {'id': 'uk-news/2023/feb/15/lucy-letby-nurse-accused-of-killing-babies-entered-grand-national-on-day-of-attempted',
   'type': 'article',
   'sectionId

In [60]:
# The dictionary under response is what matters and has a few keys with associated values..
guardian_data.keys()

dict_keys(['status', 'userTier', 'total', 'startIndex', 'pageSize', 'currentPage', 'pages', 'orderBy', 'results'])

Whilst the other keys have useful information for later, for now `results` is key that contains the news results we want...

In [62]:
guardian_data['results']

[{'id': 'politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
  'type': 'article',
  'sectionId': 'politics',
  'sectionName': 'Politics',
  'webPublicationDate': '2023-02-15T15:54:11Z',
  'webTitle': 'Brussels contacts Sweden over plans to deport British woman with Alzheimer’s',
  'webUrl': 'https://www.theguardian.com/politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
  'apiUrl': 'https://content.guardianapis.com/politics/2023/feb/15/european-commission-contacts-sweden-plans-deport-british-woman-brexit',
  'isHosted': False,
  'pillarId': 'pillar/news',
  'pillarName': 'News'},
 {'id': 'uk-news/2023/feb/15/lucy-letby-nurse-accused-of-killing-babies-entered-grand-national-on-day-of-attempted',
  'type': 'article',
  'sectionId': 'uk-news',
  'sectionName': 'UK news',
  'webPublicationDate': '2023-02-15T15:51:15Z',
  'webTitle': 'Nurse accused of killing babies won Grand National win on day of attempted m

In [63]:
# As a list of dictionaries Pandas is able to restructure this information into a table

results  = pd.DataFrame(guardian_data['results'])
results

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName
0,politics/2023/feb/15/european-commission-conta...,article,politics,Politics,2023-02-15T15:54:11Z,Brussels contacts Sweden over plans to deport ...,https://www.theguardian.com/politics/2023/feb/...,https://content.guardianapis.com/politics/2023...,False,pillar/news,News
1,uk-news/2023/feb/15/lucy-letby-nurse-accused-o...,article,uk-news,UK news,2023-02-15T15:51:15Z,Nurse accused of killing babies won Grand Nati...,https://www.theguardian.com/uk-news/2023/feb/1...,https://content.guardianapis.com/uk-news/2023/...,False,pillar/news,News
2,football/2023/feb/15/football-daily-email-chel...,article,football,Football,2023-02-15T15:45:15Z,"Graham Potter, time and Chelsea’s 25 years of ...",https://www.theguardian.com/football/2023/feb/...,https://content.guardianapis.com/football/2023...,False,pillar/sport,Sport
3,politics/live/2023/feb/15/ben-wallace-defence-...,liveblog,politics,Politics,2023-02-15T15:44:01Z,Nicola Sturgeon denies ‘short-term pressures’ ...,https://www.theguardian.com/politics/live/2023...,https://content.guardianapis.com/politics/live...,False,pillar/news,News
4,us-news/live/2023/feb/15/biden-debt-limit-hous...,liveblog,us-news,US news,2023-02-15T15:43:55Z,Biden seeks to turn voters against Republicans...,https://www.theguardian.com/us-news/live/2023/...,https://content.guardianapis.com/us-news/live/...,False,pillar/news,News
5,world/live/2023/feb/15/russia-ukraine-war-live...,liveblog,world,World news,2023-02-15T15:35:52Z,Russia-Ukraine war live: Luhansk governor deni...,https://www.theguardian.com/world/live/2023/fe...,https://content.guardianapis.com/world/live/20...,False,pillar/news,News
6,business/live/2023/feb/15/uk-inflation-energy-...,liveblog,business,Business,2023-02-15T15:33:19Z,UK inflation falls from 10.5% to 10.1% amid on...,https://www.theguardian.com/business/live/2023...,https://content.guardianapis.com/business/live...,False,pillar/news,News
7,world/2023/feb/15/china-provinces-spent-almost...,article,world,World news,2023-02-15T15:26:43Z,China’s provinces spent almost £43bn on Covid ...,https://www.theguardian.com/world/2023/feb/15/...,https://content.guardianapis.com/world/2023/fe...,False,pillar/news,News
8,uk-news/2023/feb/15/further-strike-action-in-e...,article,uk-news,UK news,2023-02-15T15:19:19Z,Further strike action in England looks ‘inevit...,https://www.theguardian.com/uk-news/2023/feb/1...,https://content.guardianapis.com/uk-news/2023/...,False,pillar/news,News
9,uk-news/2023/feb/15/titanic-new-footage-wrecka...,article,uk-news,UK news,2023-02-15T15:19:14Z,Haunting new footage of Titanic wreckage to be...,https://www.theguardian.com/uk-news/2023/feb/1...,https://content.guardianapis.com/uk-news/2023/...,False,pillar/news,News


In [64]:
# To summarise the process
parameters = {'api-key':GUARDIAN_KEY}
response = requests.get(API_ENDPOINT, params=parameters)
guardian_data = response.json()['response']
results = pd.DataFrame(guardian_data['results'])

results

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName
0,world/live/2023/feb/15/russia-ukraine-war-live...,liveblog,world,World news,2023-02-15T16:21:19Z,Russia-Ukraine war live: Luhansk governor deni...,https://www.theguardian.com/world/live/2023/fe...,https://content.guardianapis.com/world/live/20...,False,pillar/news,News
1,us-news/live/2023/feb/15/biden-debt-limit-hous...,liveblog,us-news,US news,2023-02-15T16:21:13Z,Biden seeks to turn voters against Republicans...,https://www.theguardian.com/us-news/live/2023/...,https://content.guardianapis.com/us-news/live/...,False,pillar/news,News
2,politics/live/2023/feb/15/ben-wallace-defence-...,liveblog,politics,Politics,2023-02-15T16:18:12Z,Nicola Sturgeon denies ‘short-term pressures’ ...,https://www.theguardian.com/politics/live/2023...,https://content.guardianapis.com/politics/live...,False,pillar/news,News
3,politics/2023/feb/15/nicola-sturgeon-in-her-ow...,article,politics,Politics,2023-02-15T16:13:01Z,Nicola Sturgeon in her own words: key moments ...,https://www.theguardian.com/politics/2023/feb/...,https://content.guardianapis.com/politics/2023...,False,pillar/news,News
4,commentisfree/2023/feb/15/keir-starmer-jeremy-...,article,commentisfree,Opinion,2023-02-15T16:12:12Z,Starmer is right to stop Corbyn standing for L...,https://www.theguardian.com/commentisfree/2023...,https://content.guardianapis.com/commentisfree...,False,pillar/opinion,Opinion
5,sport/2023/feb/15/nothing-will-stop-me-lewis-h...,article,sport,Sport,2023-02-15T16:11:58Z,‘Nothing will stop me’: Lewis Hamilton defiant...,https://www.theguardian.com/sport/2023/feb/15/...,https://content.guardianapis.com/sport/2023/fe...,False,pillar/sport,Sport
6,uk-news/2023/feb/15/lucy-letby-nurse-accused-o...,article,uk-news,UK news,2023-02-15T16:09:16Z,Lucy Letby: nurse won Grand National bet on da...,https://www.theguardian.com/uk-news/2023/feb/1...,https://content.guardianapis.com/uk-news/2023/...,False,pillar/news,News
7,business/2023/feb/15/ftse-100-hits-8000-points...,article,business,Business,2023-02-15T16:07:45Z,"FTSE 100 hits 8,000 points for first time as r...",https://www.theguardian.com/business/2023/feb/...,https://content.guardianapis.com/business/2023...,False,pillar/news,News
8,sport/2023/feb/15/brendon-mccullum-embraces-ru...,article,sport,Sport,2023-02-15T16:00:15Z,Brendon McCullum embraces rugby roots before N...,https://www.theguardian.com/sport/2023/feb/15/...,https://content.guardianapis.com/sport/2023/fe...,False,pillar/sport,Sport
9,politics/2023/feb/15/european-commission-conta...,article,politics,Politics,2023-02-15T15:54:11Z,Brussels contacts Sweden over plans to deport ...,https://www.theguardian.com/politics/2023/feb/...,https://content.guardianapis.com/politics/2023...,False,pillar/news,News


## Customising your request with parameters
To customise our query we simply need to add to or adjust the parameters we pass to our request.

### Query
The search query is the primary way to filter our results.

In [65]:
parameters = {'api-key':GUARDIAN_KEY,
              'q':'crime'}

response = requests.get(API_ENDPOINT, params=parameters)
guardian_data = response.json()['response']
results = pd.DataFrame(guardian_data['results'])

results

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName
0,film/2023/jan/08/the-enforcer-review-antonio-b...,article,film,Film,2023-01-08T13:00:48Z,The Enforcer review – thuddingly predictable c...,https://www.theguardian.com/film/2023/jan/08/t...,https://content.guardianapis.com/film/2023/jan...,False,pillar/arts,Arts
1,books/2023/jan/20/the-best-recent-and-thriller...,article,books,Books,2023-01-20T12:00:37Z,The best recent crime and thrillers – review r...,https://www.theguardian.com/books/2023/jan/20/...,https://content.guardianapis.com/books/2023/ja...,False,pillar/arts,Arts
2,books/2023/jan/24/the-best-recent-and-thriller...,article,books,Books,2023-01-24T09:00:29Z,The best recent crime and thriller writing – r...,https://www.theguardian.com/books/2023/jan/24/...,https://content.guardianapis.com/books/2023/ja...,False,pillar/arts,Arts
3,society/2023/jan/23/assisted-dying-should-be-r...,article,society,Society,2023-01-23T16:19:32Z,Assisted dying should be a right – not a crime...,https://www.theguardian.com/society/2023/jan/2...,https://content.guardianapis.com/society/2023/...,False,pillar/news,News
4,world/2023/feb/08/contempt-ruling-against-envi...,article,world,World news,2023-02-08T18:20:22Z,Contempt ruling against environmental proteste...,https://www.theguardian.com/world/2023/feb/08/...,https://content.guardianapis.com/world/2023/fe...,False,pillar/news,News
5,news/2022/dec/07/inside-guardian-weekly-9-dece...,article,news,News,2022-12-07T09:00:36Z,Crime scene: Inside the 9 December Guardian We...,https://www.theguardian.com/news/2022/dec/07/i...,https://content.guardianapis.com/news/2022/dec...,False,pillar/news,News
6,books/2022/dec/03/the-best-and-thriller-books-...,article,books,Books,2022-12-03T09:00:42Z,The best crime and thriller books of 2022,https://www.theguardian.com/books/2022/dec/03/...,https://content.guardianapis.com/books/2022/de...,False,pillar/arts,Arts
7,uk-news/2023/feb/15/lancashire-police-no-evide...,article,uk-news,UK news,2023-02-15T13:20:52Z,Lancashire police maintain ‘no evidence’ of cr...,https://www.theguardian.com/uk-news/2023/feb/1...,https://content.guardianapis.com/uk-news/2023/...,False,pillar/news,News
8,books/2022/nov/18/the-best-recent-and-thriller...,article,books,Books,2022-11-18T12:00:27Z,The best recent crime and thrillers – review r...,https://www.theguardian.com/books/2022/nov/18/...,https://content.guardianapis.com/books/2022/no...,False,pillar/arts,Arts
9,commentisfree/2023/jan/22/can-victims-of-hope-...,article,commentisfree,Opinion,2023-01-22T07:30:29Z,Can victims of crime hope for justice from a ...,https://www.theguardian.com/commentisfree/2023...,https://content.guardianapis.com/commentisfree...,False,pillar/opinion,Opinion


In [66]:
response.url

'http://content.guardianapis.com/search?api-key=***REMOVED***&q=crime'

Queries can be more than one word. The Guardian API documentation explains a number of ways you might adjust your query.
- 'Crime AND Prison' - Search for articles where both the terms 'crime' and 'prison' are used.
- 'Crime OR Prison' for either term. - Search for articles where either 'crime' or 'prison' are used.
- '"Criminal justice"' - Using quote marks to search for a phrase.
- 'debate AND NOT immigration' - Search for articles that use the term debate, but not the term immigration.

See the [Guardian API documentation](https://open-platform.theguardian.com/documentation/) for more options.

In [72]:
# Phrases require an extra step because of the way requests works.

parameters = {'api-key':GUARDIAN_KEY,
              'q':'"human rights"'}

response = requests.get(API_ENDPOINT, params=parameters)
guardian_data = response.json()['response']
results = pd.DataFrame(guardian_data['results'])
results

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName
0,global-development/2023/feb/10/uk-accused-whit...,article,global-development,Global development,2023-02-10T13:15:53Z,UK accused of ‘dangerous whitewash’ over Bahra...,https://www.theguardian.com/global-development...,https://content.guardianapis.com/global-develo...,False,pillar/news,News
1,law/2022/dec/08/watchdog-criticises-uk-ministe...,article,law,Law,2022-12-08T23:00:07Z,Watchdog criticises UK ministers’ ‘antagonism’...,https://www.theguardian.com/law/2022/dec/08/wa...,https://content.guardianapis.com/law/2022/dec/...,False,pillar/news,News
2,us-news/2023/feb/14/biden-withdraws-nominee-hu...,article,us-news,US news,2023-02-15T00:07:16Z,Biden drops candidate’s nomination to human ri...,https://www.theguardian.com/us-news/2023/feb/1...,https://content.guardianapis.com/us-news/2023/...,False,pillar/news,News
3,world/2023/feb/08/uganda-condemned-decision-cl...,article,global-development,Global development,2023-02-08T12:40:31Z,Uganda condemned for ‘shameful’ decision to cl...,https://www.theguardian.com/world/2023/feb/08/...,https://content.guardianapis.com/world/2023/fe...,False,pillar/news,News
4,law/2023/jan/25/rishi-sunak-told-to-ditch-plan...,article,law,Law,2023-01-25T08:44:42Z,Rishi Sunak told to ditch plans to overhaul hu...,https://www.theguardian.com/law/2023/jan/25/ri...,https://content.guardianapis.com/law/2023/jan/...,False,pillar/news,News
5,australia-news/2023/feb/15/queenslands-strict-...,article,australia-news,Australia news,2023-02-14T14:00:18Z,Human Rights Act could be critical in divisive...,https://www.theguardian.com/australia-news/202...,https://content.guardianapis.com/australia-new...,False,pillar/news,News
6,world/2022/dec/14/new-zealands-emergency-housi...,article,world,World news,2022-12-14T03:00:36Z,New Zealand’s emergency housing system breache...,https://www.theguardian.com/world/2022/dec/14/...,https://content.guardianapis.com/world/2022/de...,False,pillar/news,News
7,global-development/2023/jan/12/uk-risks-being-...,article,global-development,Global development,2023-01-12T12:32:27Z,UK risks being listed as a ‘human rights abuse...,https://www.theguardian.com/global-development...,https://content.guardianapis.com/global-develo...,False,pillar/news,News
8,global-development/2023/jan/05/volker-turk-un-...,article,global-development,Global development,2023-01-05T06:30:13Z,Volker Türk: the man charged with protecting t...,https://www.theguardian.com/global-development...,https://content.guardianapis.com/global-develo...,False,pillar/news,News
9,environment/2022/nov/15/the-egyptian-human-rig...,article,environment,Environment,2022-11-15T10:54:00Z,The Egyptian human rights activists unable to ...,https://www.theguardian.com/environment/2022/n...,https://content.guardianapis.com/environment/2...,False,pillar/news,News


### Additional Filters
Other useful filters that might be of value when narrowing down your search...

In [79]:


parameters = {'api-key':GUARDIAN_KEY,
              'q':'"human rights"',
              'page-size':10, # controls how many results you get per request - max 200
              'production-office':'uk', # what
              'lang':'en',
              'from-date':'2023-01-20',
              'to-date':'2023-01-30'
              }

response = requests.get(API_ENDPOINT, params=parameters)
guardian_data = response.json()['response']
results = pd.DataFrame(guardian_data['results'])
results

KeyError: 'results'

In [80]:
guardian_data

{'status': 'error',
 'message': 'page-size must be an integer between 0 and 200'}

In [75]:
results.loc[0,'tags']

[{'id': 'global-development/global-development',
  'type': 'keyword',
  'sectionId': 'global-development',
  'sectionName': 'Global development',
  'webTitle': 'Global development',
  'webUrl': 'https://www.theguardian.com/global-development/global-development',
  'apiUrl': 'https://content.guardianapis.com/global-development/global-development',
  'references': []},
 {'id': 'global-development/human-rights',
  'type': 'keyword',
  'sectionId': 'global-development',
  'sectionName': 'Global development',
  'webTitle': 'Human rights',
  'webUrl': 'https://www.theguardian.com/global-development/human-rights',
  'apiUrl': 'https://content.guardianapis.com/global-development/human-rights',
  'references': []},
 {'id': 'world/bahrain',
  'type': 'keyword',
  'sectionId': 'world',
  'sectionName': 'World news',
  'webTitle': 'Bahrain',
  'webUrl': 'https://www.theguardian.com/world/bahrain',
  'apiUrl': 'https://content.guardianapis.com/world/bahrain',
  'references': []},
 {'id': 'politics/

In [40]:
query = 'royal family'
page_size = 50
section = 'uk-news'
format_query(query)

'royal%20family'

In [35]:
params = {'api-key':GUARDIAN_KEY,
          'q':query,
          'page-size':page_size,
          'show-tags':'keyword',
          'tag':'type/article',
          'order-by':'newest',
          'page':1,
          'section':section,
          'show-fields':'wordcount,body,byline',
          'from-date':'2023-02-10'
          }

In [36]:
res = requests.get(API_ENDPOINT, params=params,)
res.url

'http://content.guardianapis.com/search?api-key=***REMOVED***&q=royal%2520family&page-size=50&show-tags=keyword&tag=type%2Farticle&order-by=newest&page=1&section=uk-news&show-fields=wordcount%2Cbody%2Cbyline'

In [19]:
res.json()

{'response': {'status': 'ok',
  'userTier': 'developer',
  'total': 3690,
  'startIndex': 1,
  'pageSize': 50,
  'currentPage': 1,
  'pages': 74,
  'orderBy': 'newest',
  'results': [{'id': 'uk-news/2023/feb/08/mary-queen-of-scots-prison-letters-finally-decoded',
    'type': 'article',
    'sectionId': 'uk-news',
    'sectionName': 'UK news',
    'webPublicationDate': '2023-02-08T06:00:10Z',
    'webTitle': 'Mary, Queen of Scots prison letters finally decoded',
    'webUrl': 'https://www.theguardian.com/uk-news/2023/feb/08/mary-queen-of-scots-prison-letters-finally-decoded',
    'apiUrl': 'https://content.guardianapis.com/uk-news/2023/feb/08/mary-queen-of-scots-prison-letters-finally-decoded',
    'fields': {'byline': 'Mabel Banfield-Nwachi and Hannah Devlin',
     'body': '<p>A team of codebreakers have cracked secret coded messages in letters written by Mary, Queen of Scots while she was imprisoned by her cousin Queen Elizabeth I, in what experts have said is the most significant dis

In [25]:
def flatten_nested_dicts(df):
    dicts = df.to_dict(orient='records')
    flattened = pd.json_normalize(dicts)
    return flattened

flatten_nested_dicts(pd.DataFrame(res.json()['response']['results']))

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,tags,isHosted,pillarId,pillarName,fields.byline,fields.body,fields.wordcount
0,uk-news/2023/feb/08/mary-queen-of-scots-prison...,article,uk-news,UK news,2023-02-08T06:00:10Z,"Mary, Queen of Scots prison letters finally de...",https://www.theguardian.com/uk-news/2023/feb/0...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/uk', 'type': 'keyword', 'sectionId...",False,pillar/news,News,Mabel Banfield-Nwachi and Hannah Devlin,<p>A team of codebreakers have cracked secret ...,501
1,uk-news/2023/feb/04/nurses-offer-to-call-off-s...,article,uk-news,UK news,2023-02-04T17:00:08Z,Nurses offer to call off strikes if Sunak matc...,https://www.theguardian.com/uk-news/2023/feb/0...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk-news/industrial-action', 'type': '...",False,pillar/news,News,Toby Helm Political editor,<p>The head of the UK’s biggest nursing union ...,488
2,uk-news/2023/feb/04/royal-navy-investigation-d...,article,uk-news,UK news,2023-02-04T13:27:25Z,Royal Navy opens investigation into contaminat...,https://www.theguardian.com/uk-news/2023/feb/0...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/royal-navy', 'type': 'keyword', 's...",False,pillar/news,News,Tom Ambrose,<p>The Royal Navy has announced an investigati...,400
3,uk-news/2023/feb/03/royal-navy-sailors-in-hosp...,article,uk-news,UK news,2023-02-03T23:00:08Z,Royal Navy sailors in hospital after ‘wrong ch...,https://www.theguardian.com/uk-news/2023/feb/0...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/royal-navy', 'type': 'keyword', 's...",False,pillar/news,News,Jane Clinton,<p>Sailors are being treated in hospital after...,358
4,uk-news/2023/feb/01/royal-navy-orders-investig...,article,uk-news,UK news,2023-02-01T00:02:37Z,Royal Navy orders investigation into nuclear s...,https://www.theguardian.com/uk-news/2023/feb/0...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/royal-navy', 'type': 'keyword', 's...",False,pillar/news,News,Joe Middleton,<p>The Royal Navy has ordered an urgent invest...,415
5,uk-news/2023/jan/31/met-officers-did-not-exami...,article,uk-news,UK news,2023-01-31T07:00:41Z,Met officers did not examine if spying was jus...,https://www.theguardian.com/uk-news/2023/jan/3...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/uk', 'type': 'keyword', 'sectionId...",False,pillar/news,News,Rob Evans,<p>None of the senior police officers in charg...,596
6,uk-news/2023/jan/29/prince-andrew-urged-to-cha...,article,uk-news,UK news,2023-01-29T20:05:41Z,Ex-royal aide says trying to reopen deal with ...,https://www.theguardian.com/uk-news/2023/jan/2...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/prince-andrew', 'type': 'keyword',...",False,pillar/news,News,Robert Booth Social affairs correspondent,<p>Any attempt by Prince Andrew to try to row ...,800
7,uk-news/2023/jan/28/gulf-royals-own-more-than-...,article,uk-news,UK news,2023-01-28T07:30:13Z,Gulf royals own more than £1bn of UK property ...,https://www.theguardian.com/uk-news/2023/jan/2...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'business/realestate', 'type': 'keywor...",False,pillar/news,News,Rob Davies,<p>The royal families of Gulf states including...,606
8,uk-news/2023/jan/27/theatrical-tour-tells-lgbt...,article,uk-news,UK news,2023-01-27T06:00:43Z,Theatrical tour tells LGBTQ+ history of Tower ...,https://www.theguardian.com/uk-news/2023/jan/2...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/london', 'type': 'keyword', 'secti...",False,pillar/news,News,Esther Addley,"<p>In June 1916, a 22-year-old soldier from th...",613
9,uk-news/2023/jan/23/woman-arrested-after-man-d...,article,uk-news,UK news,2023-01-23T18:05:13Z,Woman arrested after man dies in suspected mob...,https://www.theguardian.com/uk-news/2023/jan/2...,https://content.guardianapis.com/uk-news/2023/...,"[{'id': 'uk/uk', 'type': 'keywo

##### API Details
In order for people to know how to query an API (without having a helpful University class) APIs  will usually have documentation. The Guardian API is no different. If you want to understand what different parameters do, or what other options you have to explore the API, the documentation is the best place to start.

[Guardian API Documentation](https://open-platform.theguardian.com/documentation/)